# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: Well, this is a classification problem, we need to classify the students into two classes actually, one class for the students who will do fine(graduate without supervision or intervention) and do not need any intervention before they graduate, and another class will be of the students who need early intervention before they fail to graduate. So, this is a classification problem, with two classes.**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [29]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [30]:
student_data.columns

Index([u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu',
       u'Fedu', u'Mjob', u'Fjob', u'reason', u'guardian', u'traveltime',
       u'studytime', u'failures', u'schoolsup', u'famsup', u'paid',
       u'activities', u'nursery', u'higher', u'internet', u'romantic',
       u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health',
       u'absences', u'passed'],
      dtype='object')

In [31]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = sum([1 for x in student_data['passed'] if x == "yes"])

# TODO: Calculate failing students
n_failed = sum([1 for x in student_data['passed'] if x == "no"])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/n_students*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [32]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [33]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [34]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=42, stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


In [35]:
print X_train[:100]

     school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
273        1.0        0.0    0.0    1.0   17        1.0        0.0   
265        1.0        0.0    0.0    1.0   18        1.0        0.0   
64         1.0        0.0    1.0    0.0   15        0.0        1.0   
131        1.0        0.0    1.0    0.0   15        0.0        1.0   
122        1.0        0.0    1.0    0.0   16        0.0        1.0   
25         1.0        0.0    1.0    0.0   16        0.0        1.0   
191        1.0        0.0    1.0    0.0   17        0.0        1.0   
195        1.0        0.0    1.0    0.0   17        0.0        1.0   
277        1.0        0.0    0.0    1.0   18        0.0        1.0   
39         1.0        0.0    1.0    0.0   15        1.0        0.0   
9          1.0        0.0    0.0    1.0   15        0.0        1.0   
81         1.0        0.0    0.0    1.0   15        0.0        1.0   
360        0.0        1.0    1.0    0.0   18        1.0        0.0   
29         1.0      

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:** 
**1) Logistic Regression** 

**a) Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**


Toward Probabilistic Diagnosis and Understanding of Depression Based on Functional MRI Data Analysis with Logistic Group LASSO(https://www.researchgate.net/publication/275957623_Toward_Probabilistic_Diagnosis_and_Understanding_of_Depression_Based_on_Functional_MRI_Data_Analysis_with_Logistic_Group_LASSO)


**b) What are the strengths of the model; when does it perform well?**

Logistic Regression is a well behaved as far as the features are linear, or even if some functions are ot linear they can be easily converted to linear features using feature engineering.

Logistic Regression is robust to the noise, and overfitting can be prevented using L1, L2 regularization.

As Logistic Regression gives out probabilities, and based on the probabilities and the given threshold it decides to which class the instance belongs, so it can be useful in the tasks when we have to rank the instances or want to know how strongly or weakly some instance belongs to which class.

Logistic Regression can also be scaled well and used with Big Data, since it is efficient.

Logistic Regression performs well, in cases of binary classifications, and where most of the features are linear, or can be turned into linear features using feature engineering. L1 and L2 regularization can be used to avoid overfitting. 


**c) What are the weaknesses of the model; when does it perform poorly?**

The fact that logistic regression is linear, so it cannot fit non-linear data.

**d) What makes this model a good candidate for the problem, given what you know about the data?**

For classification problem, and specially binary classification problem, Logistic Regression can always serve as a baseline to start of with, because of the way it is well behaves, the way it responds to noise, and scalability, it can be futher regularized using L1 or L2 penalty. As this is a binary classification problem, I think, logistic regression can serve as a good baseline model, and most probably can perform at par with other models if tuned properly. 

**2) Support Vector Machine** 

**a) Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**

The support vector machine (SVM) algorithm (Boser et al., 1992; Vapnik, 1998) is a classification algorithm that provides state-of-the-art performance in a wide variety of application domains, including handwriting recognition, object recognition, speaker identification, face detection and text categorization (Cristianini and Shawe-Taylor, 2000). During the past three years, SVMs have been applied very broadly within the field of computational biology, to pattern recognition problems including protein remote homology detection, microarray gene expression analysis, recognition of translation start sites, functional classification of promoter regions, prediction of protein-protein interactions, and peptide identification from mass spectrometry data. (http://noble.gs.washington.edu/papers/noble_support.pdf)

**b) What are the strengths of the model; when does it perform well?**

Support Vector Machine has a parameter for regularization, which can again as in the case of logistic regression, can avoid overfitting.

Support Vector Machine has a kernal which can be changed(or rather a lot of options for kernal are available in sklearn) to fit the data in a better way. SVM can also be used to deal with data which is not linearly separable by changing the kernal. 

Support Vector Machine uses hinge loss function which punishes the classifier for mis-classification, which helps in determining the margins.

**c) What are the weaknesses of the model; when does it perform poorly?**

Support Vector Machine is fundamentally a binary classifier, and it is not very likely to perform very well on multi-class classification problems. It struggles if decision boundaries are not very well defined.

Support Vector Machine with a linear kernal is not very different than the logistic regression. 

Support Vector Machine cannot be trained very efficiently. So, it will not work well on a last training data.

**d) What makes this model a good candidate for the problem, given what you know about the data?**

The problem that we have at hand currently is a binary classification problem, it has two classes to which the students belong, SVM is a binary classifier, we use SVM after logistic regression because if the training data we have at hand is not linearly separable then we can use SVM with 'rbf' kernal to even train on the non-linear data. One of the major weaknesses of SVM is that it is hard to train over a large training set, but in this problem our training set is not large(about 300 training points), so training a SVM classifier would not be so difficult. 


**3) K-Nearest Neighbors**

**a) Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)**

One of the industrial application of K-nearest neighbor can be in recommended system. When you know the likes and dislikes of users, and you have one new user and a record of his behaviour, you would want to cluster this user to the other users who had similar likes a dislikes. It can be recommending movies on Netflix, products on amazon, of suggest videos or songs on websites like youtube.

**b) What are the strengths of the model; when does it perform well?**

One of the major strengths of k-nearest neigbors model over the other two models would be its training time, as it is instance based learning, it practically takes no time in training, with respect to the other two models we are using for this problem(Logistic Regression, and SVM). 

K-nearest neighbors is also a very simple algorithm, it can be used with different similarity measures(distance metrics).

K-nearest neighbors can also learn complex models easily because of the simplicity of this algorithm.

**c) What are the weaknesses of the model; when does it perform poorly?**

One of the major weaknesses of k-nearest neighbors model is the kind memory it would require to store all the instances of the data. In the problem we have at hand the dataset is rather small, it only has 300 instances, but if it would have been a large data set, it would require memory of the similar order in order to store the training data.

The testing time of k-nearest neighbors can be large as compared to all the other models that we are using, as all the computations are done while making predictions.

It doesn't perform so well when the number of features are large. 

**d) What makes this model a good candidate for the problem, given what you know about the data?**

K-nearest neighbors model can put up a good comparison given the other two models we are using i.e. Logistic Regression and SVM. 


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [36]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [37]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C

from sklearn import linear_model
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = linear_model.LogisticRegression(random_state=42)
clf_B = svm.SVC(random_state=42)
clf_C = KNeighborsClassifier(n_neighbors=5)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for size in [100, 200, 300]:
    train_predict(clf_A, X_train[:size], y_train[:size], X_test, y_test)

for size in [100, 200, 300]:
    train_predict(clf_B, X_train[:size], y_train[:size], X_test, y_test)

for size in [100, 200, 300]:
    train_predict(clf_C, X_train[:size], y_train[:size], X_test, y_test)    

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8671.
Made predictions in 0.0120 seconds.
F1 score for test set: 0.7068.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8211.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7391.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8512.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7500.
Training a SVC using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8354.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.8025.
Training a SVC using a training set size of 200. . .
Trained mo

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0040 seconds       |      0.0120 seconds    |     0.8671       |     0.7068      |
| 200               |    0.0040 seconds       |      0.0000 seconds    |     0.8211       |     0.7391      |
| 300               |    0.0040 seconds       |      0.0000 seconds    |     0.8512       |     0.7500      |

** Classifer 2 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0040 seconds      |      0.0040 seconds    |     0.8354       |     0.8025      |
| 200               |     0.0040 seconds      |      0.0040 seconds    |     0.8431       |     0.8105      |
| 300               |     0.0080 seconds      |      0.0030 seconds    |     0.8664       |     0.8052      |

** Classifer 3 - K-nearest neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0010 seconds       |      0.0010 seconds    |     0.8252       |     0.7586      |
| 200               |    0.0000 seconds       |      0.0000 seconds    |     0.8097       |     0.7857      |
| 300               |    0.0000 seconds       |      0.0000 seconds    |     0.8539       |     0.8138      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: Given the performance on training set and test set, and keeping factors like available data, limited resources, cost and performance in mind, from all the three models which have been trained on this dataset, k-nearest neighbors classifier seems to be the best model. In terms of F1 score, k-nearest neignors generalizes well on the test set, when compared to the other models, also one important point to note will be the fact that as we increase the number of traning points from 100 to 300, the F1 score for k-nearest neighbors increases consistently for both the test set and training set, and as there is not a lot of difference in the F1 score of test set and training when compared to Logistic Regression and SVM, which indeed is an indication that out model is not overfitting. The training and prediction time that k-nearest neighbors takes is quite less if compared to the other two models we have trained, also because k-nearest neighbors as it is an instance based learning model, and requires no training what so ever. Also the dataset only consists of around 300 instances, which is a small dataset, so k-nearest neighbors is quite efficient in making predictions.** 

**The fact that k-nearest neighbors can be trained easily on new data as there is practically no training time required for this algorithm, makes it more suitable for an organisation like school, where most probably every year new data will be available and new model must be trained based on the incoming data, or multiple training iterations be made. K-nearest neigbor can be caliberated to avoid overfitting by choosing the number of neighnors to use in making the prediction.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: We have trained a k-nearest neighbors classifier on the dataset we were given. All the training data, with all it's features was fed to the k-nearest neighbors model. As k-nearest model is an instance based learning model, so it doesn't generate a function like other candidate model we were considering like Logistic Regression and SVM, it rather stores all the traning data in the memory. So, in instance based learning the whole traning data becomes the function, to which it has to fit the incoming data points.(https://storage.googleapis.com/supplemental_media/udacityu/5446988865/Instance%20Based%20Learning.pdf) So, when to k-nearest neighbors you feed the data of some new student with all the features (in a romantic relations or not, has internet connection or not, number of failures, absences etc.) for which you don't know whether he is likely to graduate or not. The k-nearest neighbors compares it to the other students from the training data depending upon a similarity metric (d), we can define the number of neighbors (students) which should be voting in order to classify this new student. So, what it actually means is, the students who are most similar in terms of traits(features) to this new student from the training data, will vote based on their classification (they graduated or failed), and the class which dominates in the voting will be assigned to this new student. In case of a tie, there are ways in which the tie can be broken, for example by using weighted voting we can resolve ties, the student who is very similar in traits to the new student will be assigned a higher weight when it come to counting the vote.**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [56]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'n_neighbors': [1, 3, 5, 10, 20, 25], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
           weights='uniform')
Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8206.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8101.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: The models's final F1 score for training set is 0.8206 and for testing set is 0.8101, which is very close to the F1 score we got in the untuned model. **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.